# Project 8
### Try Detectron 2

Import Library

In [1]:
import os
import numpy as np
import json
from detectron2.structures import BoxMode
import pandas as pd

In [2]:
def get_cat_dict(directory,jsonfile_name):
    classes = ['Khawlahm','Khawniewping','Khawtang']
    dataset_dicts = []
    json_file = os.path.join(directory,jsonfile_name)
    with open(json_file) as f:
        json_ann_file = json.load(f)
    
    image_list = json_ann_file['images']
    annotation_list = pd.DataFrame(json_ann_file['annotations'])
    
    for image in image_list :
        record = {}
        filename = os.path.join(directory,image['file_name'])

        record['file_name'] = filename
        record['height'] = image['height']
        record['width'] = image['width']
        image_id = image['id']

        # find the annotation of this image
        annos = annotation_list[annotation_list['image_id'] == image_id]
        objs = []
        for idx,anno in  annos.iterrows() :
            obj = {
                "bbox":anno['bbox'],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation":anno['segmentation'],
                "category_id":anno['category_id'],
                'iscrowd':0
            }
            objs.append(obj)
        record['annotations'] = objs
        dataset_dicts.append(record)

    return dataset_dicts
    

Register Data Catalog

In [3]:
from detectron2.data import DatasetCatalog, MetadataCatalog
for d in ["train", "test"]:
    DatasetCatalog.register("cat_" + d, lambda d=d: get_cat_dict('cat-dataset/','result.json'))
    MetadataCatalog.get("cat_" + d).set(thing_classes=['Khawlahm','Khawniewping','Khawtang'])

In [4]:
cat_metadata = MetadataCatalog.get("cat_train")

Training The Model

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
